In [1]:
from snowflake.snowpark import Session
import pandas as pd
import requests
from snowflake.snowpark.functions import sproc

In [2]:
#Connecting to snowflake
connection_parameters = {
"account": "ld77469.uae-north.azure",
"user": "GOKS98",
"password": "March20311",
"role": "ACCOUNTADMIN",
"warehouse": "SNOWFLAKE_WAREHOUSE",
"database": "OIL_AND_GAS",
"schema": "CONFORMED"
 }

session = Session.builder.configs(connection_parameters).create()

In [4]:
session.add_packages('pandas', 'requests', 'snowflake-snowpark-python')

@sproc(session=session, is_permanent=True, stage_location='@MY_STAGE', name='sp_weather_join', replace=True)
def api_weather(session: Session) -> str:
    #Creating a Dataframe
    df_table = session.table("SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.WAREHOUSE")

    #Converting to pandas df to make manipulation easier
    df_pd = df_table.to_pandas()

    #Getting weather data from API for every zipcode
    json_list = []
    for i in df_pd['W_ZIP']:
        response = requests.get(f"https://api.weatherapi.com/v1/current.json?key=c75b271f50ea4479a7361323220211&q={i}")
        data = response.json()
        json_list.append(data)
    json_list

    #Creating a new df with weather info and zip code
    zip_code = df_pd['W_ZIP']
    data = list(zip(zip_code, json_list))
    weather_df = pd.DataFrame(data=data, columns=['W_ZIP', 'WEATHER_JSON'])
    
    #Joining weather nad warehouse tables
    snowflake_df = df_pd.merge(weather_df, how='inner', on='W_ZIP')

    snowflake_df_1 = session.create_dataframe(snowflake_df)
    #Writing table to snowflake
    snowflake_df_1.write.mode("overwrite").save_as_table("w_warehouse_weather")

    return 'Success!!'
    

The version of package snowflake-snowpark-python in the local environment is 0.12.0, which does not fit the criteria for the requirement snowflake-snowpark-python. Your UDF might not work when the package version is different between the server and your local environment


In [5]:
session.call('sp_weather_join').collect()

Failed to execute query [queryID: 01a80970-0000-0a8b-0000-000035d9d121] CALL sp_weather_join()
100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "/usr/lib/python_udf/75475333f5736f745fc7bdbab89f5473fd667011ff347d8ca7369e7931855ae7/lib/python3.8/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/usr/lib/python_udf/75475333f5736f745fc7bdbab89f5473fd667011ff347d8ca7369e7931855ae7/lib/python3.8/site-packages/urllib3/util/connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/usr/lib/python_udf/75475333f5736f745fc7bdbab89f5473fd667011ff347d8ca7369e7931855ae7/lib/python3.8/socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -5] No address associated with hostname

During handling of the above exception, another exception occurred:

Traceback 

SnowparkSQLException: (1304): 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "/usr/lib/python_udf/75475333f5736f745fc7bdbab89f5473fd667011ff347d8ca7369e7931855ae7/lib/python3.8/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/usr/lib/python_udf/75475333f5736f745fc7bdbab89f5473fd667011ff347d8ca7369e7931855ae7/lib/python3.8/site-packages/urllib3/util/connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/usr/lib/python_udf/75475333f5736f745fc7bdbab89f5473fd667011ff347d8ca7369e7931855ae7/lib/python3.8/socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -5] No address associated with hostname

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python_udf/75475333f573